## Lecture 09 SQLite Examples

Helpful here is the [SQLite Cheat Sheet](https://www.sqlitetutorial.net/sqlite-cheat-sheet/)

In [38]:
import pandas as pd
import sqlite3

In [39]:
# Create a DB and connect to it.
conn = sqlite3.connect("lec09.db")
cursor = conn.cursor()

In [40]:
cursor.execute('''
CREATE TABLE cats (
    id INTEGER PRIMARY KEY,
    name TEXT
)''')


In [41]:
# Insert into the table
cursor.execute('''INSERT INTO cats VALUES (1, "Megabyte")''')
cursor.execute('''INSERT INTO cats VALUES (2, "Meowly Cyrus")''')
cursor.execute('''INSERT INTO cats VALUES (3, "Fuzz Aldrin")''')
cursor.execute('''INSERT INTO cats VALUES (4, "Chairman Meow")''')
cursor.execute('''INSERT INTO cats VALUES (5, "Anderson Pooper")''')
cursor.execute('''INSERT INTO cats VALUES (6, "Gigabyte")''')
conn.commit()


In [42]:
# What if we run an SQL Command?
cursor.execute('''SELECT * FROM cats''')

In [43]:
# We need to iterate over the object we get back...
for row in cursor.execute('''SELECT * FROM cats'''):
    print(row)


(1, 'Megabyte')
(2, 'Meowly Cyrus')
(3, 'Fuzz Aldrin')
(4, 'Chairman Meow')
(5, 'Anderson Pooper')
(6, 'Gigabyte')


In [44]:
# Or we can just go to Pandas from here...  
# note that we can tell Pandas what to do for an index here.
pd.read_sql_query("SELECT * FROM cats", conn, index_col="id")

,name
id,
1,Megabyte
2,Meowly Cyrus
3,Fuzz Aldrin
4,Chairman Meow
5,Anderson Pooper
6,Gigabyte


In [51]:
# This is a full up DB so we can delete stuff too..
cursor.execute('''DELETE FROM cats WHERE id == 4''')
conn.commit()
pd.read_sql_query("SELECT * FROM cats", conn, index_col="id")


,name
id,
1,Megabyte
2,Meowly Cyrus
3,Fuzz Aldrin
5,Anderson Pooper
6,Gigabyte


In [45]:
# Let's make another table for joins.
cursor.execute('''
CREATE TABLE visits (
    cat_id INTEGER PRIMARY KEY,
    date DATE
)''')

In [46]:
cursor.execute('''INSERT INTO visits VALUES (1, "02-16-2017")''')
cursor.execute('''INSERT INTO visits VALUES (2, "02-14-2017")''')
cursor.execute('''INSERT INTO visits VALUES (5, "02-03-2017")''')
conn.commit()

In [47]:
# We can do our join in Pandas
df_cats = pd.read_sql_query('''SELECT * from cats''', conn)
df_visits = pd.read_sql_query('''SELECT * from visits''', conn)
df_cats.merge(df_visits, how = "inner", 
              left_on = "id", right_on = "cat_id")


,id,name,cat_id,date
0,1,Megabyte,1,02-16-2017
1,2,Meowly Cyrus,2,02-14-2017
2,5,Anderson Pooper,5,02-03-2017


In [50]:
# Or we can directly run them on the system

result = cursor.execute('''
               SELECT 
                   *
               FROM 
                   cats, visits
               WHERE
                   cats.id == visits.cat_id
               ''')
for i in result:
    print(i)


(1, 'Megabyte', 1, '02-16-2017')
(2, 'Meowly Cyrus', 2, '02-14-2017')
(5, 'Anderson Pooper', 5, '02-03-2017')


In [62]:
# We can also use union to combine rows, even if it doesn't make sense.
result = cursor.execute('''
               SELECT 
                   *
               FROM 
                   cats
               UNION
               SELECT
                   *
               FROM
                   visits
               ''')
for i in result:
    print(i)

(1, '02-16-2017')
(1, 'Megabyte')
(2, '02-14-2017')
(2, 'Meowly Cyrus')
(3, 'Fuzz Aldrin')
(5, '02-03-2017')
(5, 'Anderson Pooper')
(6, 'Gigabyte')


In [53]:
# What if we want a different join?
df_cats.merge(df_visits, how = "left", 
              left_on = "id", right_on = "cat_id")



,id,name,cat_id,date
0,1,Megabyte,1.0,02-16-2017
1,2,Meowly Cyrus,2.0,02-14-2017
2,3,Fuzz Aldrin,NaN,NaN
3,4,Chairman Meow,NaN,NaN
4,5,Anderson Pooper,5.0,02-03-2017
5,6,Gigabyte,NaN,NaN


In [58]:
result = cursor.execute('''SELECT * FROM cats LEFT JOIN visits ON
                   cats.id == visits.cat_id''')
for i in result:
    print(i)

(1, 'Megabyte', 1, '02-16-2017')
(2, 'Meowly Cyrus', 2, '02-14-2017')
(3, 'Fuzz Aldrin', None, None)
(5, 'Anderson Pooper', 5, '02-03-2017')
(6, 'Gigabyte', None, None)


In [63]:
# What if we want a different join?
df_cats.merge(df_visits, how = "outer", 
              left_on = "id", right_on = "cat_id")



,id,name,cat_id,date
0,1,Megabyte,1.0,02-16-2017
1,2,Meowly Cyrus,2.0,02-14-2017
2,3,Fuzz Aldrin,NaN,NaN
3,4,Chairman Meow,NaN,NaN
4,5,Anderson Pooper,5.0,02-03-2017
5,6,Gigabyte,NaN,NaN


In [64]:
# Let's do an Outer Join!

result = cursor.execute('''SELECT * FROM cats FULL OUTER JOIN visits 
                    ON cats.id == visits.cat_id''')
for i in result:
    print(i)

OperationalError: RIGHT and FULL OUTER JOINs are not currently supported

## GroupBy and Aggregate in SQL.


In [65]:
# Make a new Table...
cursor.execute('''
CREATE TABLE persons (
    id INTEGER PRIMARY KEY,
    age FLOAT,
    wgt_kg FLOAT,
    hgt_cm FLOAT,
    nat_id INTEGER
    
)''')

In [66]:
# Insert into the table
cursor.execute('''INSERT INTO persons VALUES (1, 12.2, 42.3, 145.1, 1)''')
cursor.execute('''INSERT INTO persons VALUES (2, 11.0, 40.8, 143.8, 1)''')
cursor.execute('''INSERT INTO persons VALUES (3, 15.6, 65.3, 165.3, 2)''')
cursor.execute('''INSERT INTO persons VALUES (4, 35.1, 84.2, 185.8, 1)''')
cursor.execute('''INSERT INTO persons VALUES (5, 18.1, 62.2, 176.2, 3)''')
cursor.execute('''INSERT INTO persons VALUES (6, 19.6, 82.1, 180.1, 1)''')
conn.commit()

In [67]:
ret = cursor.execute('''SELECT * FROM persons''')
for row in ret:
    print(row)

(1, 12.2, 42.3, 145.1, 1)
(2, 11.0, 40.8, 143.8, 1)
(3, 15.6, 65.3, 165.3, 2)
(4, 35.1, 84.2, 185.8, 1)
(5, 18.1, 62.2, 176.2, 3)
(6, 19.6, 82.1, 180.1, 1)


In [70]:
ret = cursor.execute('''SELECT nat_id, AVG(age) as average_age
                        FROM persons
                        GROUP BY nat_id ''')
for row in ret:
    print(row)

(1, 19.475)
(2, 15.6)
(3, 18.1)
